In [1]:
!pip install --upgrade gensim
!pip -q install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 101.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 365 kB 5.2 MB/s 
     |████████████████████████████████| 115 kB 49.5 MB/s 
     |████████████████████████████████| 120 kB 53.4 MB/s 
     |████████████████████████████████| 212 kB 62.0 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 


In [2]:
import gensim 
gensim.__version__

'4.2.0'

In [3]:
from google.colab import drive
drive.mount('/content/drive')
ROOT = '/content/drive/MyDrive/111-1成大課程/ML/'
ModelPath = f'{ROOT}/agnews_fastex.model'

Mounted at /content/drive


In [4]:
from collections import defaultdict
import numpy as np 
import re
from datasets import list_datasets, load_dataset
agnews_dataset = load_dataset('ag_news')
trainset, testset = agnews_dataset['train'], agnews_dataset['test']

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# Configs
seed = 666
sg = 0 # CBOW 
window_size = 3
vector_size = 32
min_count = 1
workers = 8 
epochs = 5 
batch_words = 10000

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from gensim.models import word2vec, fasttext
from torchtext.data import get_tokenizer
from nltk.corpus import stopwords

def clean(x):
    text,label = x['text'], x['label']
    return {'text': re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9\\]',' ',text),  # \\ for \ 
          'label':label} 
trainset = trainset.map(clean)
testset = testset.map(clean)
tokenizer = get_tokenizer("basic_english")
stopwords = set(stopwords.words('english'))

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [8]:
class IterableTexts:
    def __iter__(self):
        for dataset in [trainset, testset]:
            for data in dataset:
                text = data['text']
                yield list(x.lower() for x in tokenizer(text) if x.lower() not in stopwords)

In [9]:
model = fasttext.FastText(
    sentences = IterableTexts(),
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words, 
)

In [10]:
model.save(ModelPath)

In [11]:
len(model.wv)

63611

In [ ]:
model.wv.key_to_index